In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

/Users/admin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [6]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [7]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev = 0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev = 0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [8]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev = 0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev = 0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [9]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [14]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [15]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size = (batch_size, n_noise))

In [16]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [18]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

In [20]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [21]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list = D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list = G_var_list)

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

In [ ]:
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict = {X:batch_xs, Z:noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict = {Z:noise})
        
    print('epoch:', '%04d' % epoch,
         'D loss: {:.4}'.format(loss_val_D),
         'G loss: {:.4}'.format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict = {Z:noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize = (sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
            
        plt.savefig('sample/{}.png'.format(str(epoch).zfill(3)), bbox_inches = 'tight')
        plt.close(fig)
        
print('complete')

epoch: 0000 D loss: -0.1803 G loss: -3.09
epoch: 0001 D loss: -0.3833 G loss: -1.697
epoch: 0002 D loss: -0.3959 G loss: -2.063
epoch: 0003 D loss: -0.2491 G loss: -2.598
epoch: 0004 D loss: -0.1297 G loss: -3.355
epoch: 0005 D loss: -0.2349 G loss: -2.957
epoch: 0006 D loss: -0.195 G loss: -3.415
epoch: 0007 D loss: -0.214 G loss: -2.981
epoch: 0008 D loss: -0.3591 G loss: -2.63
epoch: 0009 D loss: -0.2957 G loss: -2.84
epoch: 0010 D loss: -0.4768 G loss: -2.638
epoch: 0011 D loss: -0.3874 G loss: -2.572
epoch: 0012 D loss: -0.4647 G loss: -2.58
epoch: 0013 D loss: -0.4099 G loss: -2.613
epoch: 0014 D loss: -0.5415 G loss: -2.412
epoch: 0015 D loss: -0.2955 G loss: -2.64
epoch: 0016 D loss: -0.6409 G loss: -2.222
epoch: 0017 D loss: -0.2806 G loss: -3.068
epoch: 0018 D loss: -0.567 G loss: -2.365
epoch: 0019 D loss: -0.3907 G loss: -2.528
epoch: 0020 D loss: -0.6426 G loss: -2.421
epoch: 0021 D loss: -0.3941 G loss: -2.478
epoch: 0022 D loss: -0.4532 G loss: -2.376
epoch: 0023 D loss: